In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import * 
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pathlib

In [2]:
IMG_HEIGHT = 180
IMG_WIDTH = 180

In [3]:
data_dir_train = pathlib.Path('C:\\Users\\charm\\Documents\\GitHub\Sign-Language-Translator\\DATA\\asl_alphabet\\asl_alphabet_train')
data_dir_test = pathlib.Path('C:\\Users\\charm\\Documents\\GitHub\Sign-Language-Translator\\DATA\\asl_alphabet\\asl_alphabet_test')
# train = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train, image_size=(IMG_WIDTH, IMG_HEIGHT))
test = tf.keras.preprocessing.image_dataset_from_directory(data_dir_test, image_size=(IMG_WIDTH, IMG_HEIGHT))

Found 28 files belonging to 28 classes.


In [4]:
EPOCHS = 10
BATCH_SIZE = 32
FILTERS = 32
POOL_SIZE = (2,2)
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
UNITS = 128

In [5]:
train = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 87000 files belonging to 29 classes.
Using 69600 files for training.


In [6]:
val = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [7]:
class_names = train.class_names

In [8]:
train = train.map(lambda x,y: (float(x)/255, y))
val = val.map(lambda x,y: (float(x)/255, y))
test = test.map(lambda x,y: (float(x)/255, y))

In [9]:
def CNN():
    input = keras.layers.Input(shape=INPUT_SHAPE)
    conv1 = keras.layers.Conv2D(FILTERS, (3,3), input_shape=INPUT_SHAPE, activation='relu')(input)
    pool1 = keras.layers.MaxPool2D(pool_size=POOL_SIZE)(conv1)
    conv2 = keras.layers.Conv2D(FILTERS*2, (3,3), input_shape=INPUT_SHAPE, activation='relu')(pool1)
    pool2 = keras.layers.MaxPool2D(pool_size=POOL_SIZE)(conv2)
    conv3 = keras.layers.Conv2D(FILTERS*3, (3,3), input_shape=INPUT_SHAPE, activation='relu')(pool2)
    pool3 = keras.layers.MaxPool2D(pool_size=POOL_SIZE)(conv3)
    flatten = keras.layers.Flatten()(pool2)
    fc1 = keras.layers.Dense(units=UNITS, activation='relu')(flatten)
    d1 = keras.layers.Dropout(rate=0.1)(fc1)
    fc2 = keras.layers.Dense(units=UNITS, activation='relu')(d1)
    d2 = keras.layers.Dropout(rate=0.1)(fc2)
    output = keras.layers.Dense(len(class_names), activation='relu')(d2)

    return Model(inputs=input, outputs=output)

In [10]:
model = CNN()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 118336)            0         
_________________________________________________________________
dense (Dense)                (None, 128)              

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
history = model.fit(
  train,
  validation_data=val,
  epochs=EPOCHS
)

Epoch 1/10
   9/2175 [..............................] - ETA: 34:38 - loss: 4.2238 - accuracy: 0.0417

In [8]:
model.save("Models/CNN1.h5")

NameError: name 'model' is not defined